## Speech to text to classify

* For this task we would need a text-to-speech model to convert your voice samples into text
* The converted texts are then given to a zero shot text classification model
    * which also takes your class queries inputs
* The zero shot classifier throws out a class probability score, which you can use as your relevance score



Thankfully you dont have to do anything from scratch, all the models you need along with pretrained weights are available in huggingface-transformers library

before install transformers library you would need pytorch(also torchaudio), install using the instructions here : https://pytorch.org/get-started/locally/

Please install the library, by following the instructions here : https://huggingface.co/docs/transformers/installation

once you have these two, you are good to go



In [1]:
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration, pipeline
import torchaudio

model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr") # model for speech to text
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr") # pre-processor for speech to text model

aud = torchaudio.load("test_record.wav") # Your audio file
arr = torchaudio.functional.resample(aud[0], orig_freq=aud[1], new_freq=16000) # For some reason ubuntu records at 8000 sample resolution, the model was trained on 16000


In [2]:
inputs = processor(arr.squeeze(), sampling_rate = 16000, return_tensors = "pt") #Apply the preprocessing to your input

generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"]) # make predictions

transcription = processor.batch_decode(generated_ids, skip_special_tokens = True)[0] # Post process the predictions to readable text

/home/venky/.conda/envs/text/lib/python3.10/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [3]:
pipe = pipeline(model="facebook/bart-large-mnli") # loading the zero-shot text classification pipeline

pips = pipe(transcription, candidate_labels = ["AI", "Detection"], multi_label = True) # make predictions, candidate_labels, are you probable classes

(…)e.co/api/models/facebook/bart-large-mnli:   0%|          | 0.00/9.18k [00:00<?, ?B/s]

In [4]:
print(pips) # your predictions

{'sequence': 'if the detective replied that i used to infer my duty at once', 'labels': ['Detection', 'AI'], 'scores': [0.9030801653862, 0.21847498416900635]}
